In [2]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

### ● 不動産の一つの物件から情報を取得する。

In [3]:
#suumo新宿区
base_url = "https://suumo.jp/chintai/tokyo/sc_shinjuku/"
response = requests.get(base_url)
soup     = BeautifulSoup(response.content,"lxml")
items    = soup.find(class_="cassetteitem")

In [4]:
# スクレイピングした HTML情報出力
f = open("test-ishida-1.txt", "w")
f.write(str(items))
f.close()

In [52]:
#検証として一つずつやってみる



In [5]:
#動いたらここへ
property_name     = items.find(class_="cassetteitem_content-title").get_text() if items.find(class_="cassetteitem_content_title") else None
category          = items.find(class_="cassetteitem_content-label").span.get_text() if items.find(class_="cassetteitem_content-label") else None
address           = items.find(class_="cassetteitem_detail-col1").get_text() if items.find(class_="cassetteitem_detail-col1") else None
nearest_stations  = [station.get_text() for station in items.find_all(class_="cassetteitem_detail-text")] if items.find_all(class_="cassetteitem_detail-text") else None
construction_info = items.find(class_="cassetteitem_detail-col3").find_all("div") if items.find(class_="cassetteitem_detail-col3") else None

#？？lenを入れている意味、floorsの方は>1である理由
years_since_const = construction_info[0].get_text() if construction_info and len(construction_info) > 0 else None
number_of_floors  = construction_info[1].get_text() if construction_info and len(construction_info) > 1 else None

#？？なぜセレクトワンを使うのか？
floor_number_td   = items.select_one("tr.js-cassette_link .cassetteitem_other-col03")
floor_number      = floor_number_td.get_text(strip=True) if floor_number_td else None

rent_info         = items.select_one(".cassetteitem_other tbody .js-cassette_link")
rent_admin_fee    = " / ".join([item.get_text(strip=True) for item in rent_info.select(".cassetteitem_price--rent, .cassetteitem_price--administration")]) if rent_info else None
deposit_gratuity  = " / ".join([price.get_text(strip=True) for price in rent_info.select(".cassetteitem_price--deposit, .cassetteitem_price--gratuity")]) if rent_info else None
layout_total_area = " / ".join([detail.get_text(strip=True) for detail in rent_info.select(".cassetteitem_madori, .cassetteitem_menseki")]) if rent_info else None



In [6]:
print(construction_info)

[<div>築12年</div>, <div>12階建</div>]


In [7]:
# 各物件情報の取得
property_name     = items.find(class_="cassetteitem_content-title").get_text() if items.find(class_="cassetteitem_content-title") else None
category          = items.find(class_="cassetteitem_content-label").span.get_text() if items.find(class_="cassetteitem_content-label") else None
address           = items.find(class_="cassetteitem_detail-col1").get_text() if items.find(class_="cassetteitem_detail-col1") else None
nearest_stations  = [station.get_text() for station in items.find_all(class_="cassetteitem_detail-text")] if items.find_all(class_="cassetteitem_detail-text") else None
construction_info = items.find(class_="cassetteitem_detail-col3").find_all("div") if items.find(class_="cassetteitem_detail-col3") else None
years_since_const = construction_info[0].get_text() if construction_info and len(construction_info) > 0 else None
number_of_floors  = construction_info[1].get_text() if construction_info and len(construction_info) > 1 else None
floor_number_td   = items.select_one("tr.js-cassette_link .cassetteitem_other-col03")
floor_number      = floor_number_td.get_text(strip=True) if floor_number_td else None
rent_info         = items.select_one(".cassetteitem_other tbody .js-cassette_link")
rent_admin_fee    = " / ".join([item.get_text(strip=True) for item in rent_info.select(".cassetteitem_price--rent, .cassetteitem_price--administration")]) if rent_info else None
deposit_gratuity  = " / ".join([price.get_text(strip=True) for price in rent_info.select(".cassetteitem_price--deposit, .cassetteitem_price--gratuity")]) if rent_info else None
layout_total_area = " / ".join([detail.get_text(strip=True) for detail in rent_info.select(".cassetteitem_madori, .cassetteitem_menseki")]) if rent_info else None


# 各物件情報の表示
print("物件名称 (Property Name):", property_name)
print("カテゴリー (Category):", category)
print("住所 (Address):", address)
print("最寄り駅 (Nearest Stations):", nearest_stations)
print("築年数 (Years Since Construction):", years_since_const)
print("階建 (Number of Floors):", number_of_floors)
print("階数 (Floor Number):", floor_number)
print("賃料/管理費 (Rent/Administration Fee):", rent_admin_fee)
print("敷金/礼金 (Deposit/Key Money):", deposit_gratuity)
print("間取り/占有面積 (Layout/Total Area):", layout_total_area)

物件名称 (Property Name): 東京メトロ南北線 市ケ谷駅 12階建 築12年
カテゴリー (Category): 賃貸マンション
住所 (Address): 東京都新宿区市谷本村町
最寄り駅 (Nearest Stations): ['東京メトロ南北線/市ケ谷駅 歩5分', 'ＪＲ総武線/四ツ谷駅 歩7分', '都営新宿線/曙橋駅 歩10分']
築年数 (Years Since Construction): 築12年
階建 (Number of Floors): 12階建
階数 (Floor Number): None
賃料/管理費 (Rent/Administration Fee): 11.2万円 / 8000円
敷金/礼金 (Deposit/Key Money): 11.2万円 / 11.2万円
間取り/占有面積 (Layout/Total Area): 1K / 25.78m2


In [8]:
# 物件画像・間取り画像・詳細URLの取得
property_image_element = items.find(class_="cassetteitem_object-item")
property_image_url = property_image_element.img["rel"] if property_image_element and property_image_element.img else None

floor_plan_image_element = items.find(class_="casssetteitem_other-thumbnail")
floor_plan_image_url = floor_plan_image_element.img["rel"] if floor_plan_image_element and floor_plan_image_element.img else None

property_link_element = items.select_one("a[href*='/chintai/jnc_']")
property_link = "https://suumo.jp" + property_link_element['href'] if property_link_element else None ## 不動産サイトから詳細URLリンクを読み解き作成

# 物件画像・間取り画像・詳細URLの表示
print("物件画像 URL (Property Image URL):", property_image_url)
print("間取り情報画像 URL (Floor Plan Image URL):", floor_plan_image_url)
print("物件リンク (Property Link):", property_link)

物件画像 URL (Property Image URL): https://img01.suumo.com/front/gazo/fr/bukken/637/100347078637/100347078637_gw.jpg
間取り情報画像 URL (Floor Plan Image URL): https://img01.suumo.com/front/gazo/fr/bukken/574/100379491574/100379491574_co.jpg
物件リンク (Property Link): https://suumo.jp/chintai/jnc_000062882287/


### ● 不動産の複数ページから情報を取得する。

In [10]:
import requests
from bs4 import BeautifulSoup

# 対象の区とそのURLのリスト
wards = {
    "新宿区": "https://suumo.jp/chintai/tokyo/sc_shinjuku/?page={}",
    "渋谷区": "https://suumo.jp/chintai/tokyo/sc_shibuya/?page={}",
    "港区": "https://suumo.jp/chintai/tokyo/sc_minato/?page={}",
    "中央区": "https://suumo.jp/chintai/tokyo/sc_chuo/?page={}",
    "世田谷区": "https://suumo.jp/chintai/tokyo/sc_setagaya/?page={}",
    "目黒区": "https://suumo.jp/chintai/tokyo/sc_meguro/?page={}",
    "千代田区": "https://suumo.jp/chintai/tokyo/sc_chiyoda/?page={}"
}

max_page = 5  # 最大ページ数
all_data = []

for ward, base_url in wards.items():
    for page in range(1, max_page + 1):
        url = base_url.format(page)
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'lxml')
        items = soup.find_all("div", {"class": "cassetteitem"})

        print("区:", ward, "page:", page, "items:", len(items))

        for item in items:
            base_data = {}
            base_data["区"] = ward
            base_data["名称"] = item.find("div", {"class": "cassetteitem_content-title"}).get_text(strip=True) if item.find("div", {"class": "cassetteitem_content-title"}) else None
            base_data["カテゴリ"] = item.find("div", {"class": "cassetteitem_content-label"}).span.get_text(strip=True) if item.find("div", {"class": "cassetteitem_content-label"}) else None
            base_data["アドレス"] = item.find("li", {"class": "cassetteitem_detail-col1"}).get_text(strip=True) if item.find("li", {"class": "cassetteitem_detail-col1"}) else None
            
            # 駅のアクセス情報をまとめて取得
            base_data["アクセス"] = ", ".join([station.get_text(strip=True) for station in item.findAll("div", {"class": "cassetteitem_detail-text"})])

            construction_info = item.find("li", {"class": "cassetteitem_detail-col3"}).find_all("div") if item.find("li", {"class": "cassetteitem_detail-col3"}) else None
            base_data["築年数"] = construction_info[0].get_text(strip=True) if construction_info and len(construction_info) > 0 else None
            base_data["構造"] = construction_info[1].get_text(strip=True) if construction_info and len(construction_info) > 1 else None

            tbodys = item.find("table", {"class": "cassetteitem_other"}).findAll("tbody")

            for tbody in tbodys:
                data = base_data.copy()
                # 階数情報の正確な取得
                floor_info = tbody.find_all("td")[2].get_text(strip=True) if len(tbody.find_all("td")) > 2 else None
                data["階数"] = floor_info
                data["家賃"] = tbody.select_one(".cassetteitem_price--rent").get_text(strip=True) if tbody.select_one(".cassetteitem_price--rent") else None
                data["管理費"] = tbody.select_one(".cassetteitem_price--administration").get_text(strip=True) if tbody.select_one(".cassetteitem_price--administration") else None
                data["敷金"] = tbody.select_one(".cassetteitem_price--deposit").get_text(strip=True) if tbody.select_one(".cassetteitem_price--deposit") else None
                data["礼金"] = tbody.select_one(".cassetteitem_price--gratuity").get_text(strip=True) if tbody.select_one(".cassetteitem_price--gratuity") else None
                data["間取り"] = tbody.select_one(".cassetteitem_madori").get_text(strip=True) if tbody.select_one(".cassetteitem_madori") else None
                data["面積"] = tbody.select_one(".cassetteitem_menseki").get_text(strip=True) if tbody.select_one(".cassetteitem_menseki") else None

                # 物件画像・間取り画像・詳細URLの取得を最後に行う
                property_image_element = item.find(class_="cassetteitem_object-item")
                data["物件画像URL"] = property_image_element.img["rel"] if property_image_element and property_image_element.img else None

                floor_plan_image_element = item.find(class_="casssetteitem_other-thumbnail")
                data["間取画像URL"] = floor_plan_image_element.img["rel"] if floor_plan_image_element and floor_plan_image_element.img else None

                property_link_element = item.select_one("a[href*='/chintai/jnc_']")
                data["物件詳細URL"] = "https://suumo.jp" +property_link_element['href'] if property_link_element else None

                all_data.append(data)

print(f"合計 {len(all_data)} 件のデータを収集しました。")

# データを確認する場合
import pandas as pd
df = pd.DataFrame(all_data)
print(df.head())


区: 新宿区 page: 1 items: 20
区: 新宿区 page: 2 items: 20
区: 新宿区 page: 3 items: 20
区: 新宿区 page: 4 items: 20
区: 新宿区 page: 5 items: 20
区: 渋谷区 page: 1 items: 20
区: 渋谷区 page: 2 items: 20
区: 渋谷区 page: 3 items: 20
区: 渋谷区 page: 4 items: 20
区: 渋谷区 page: 5 items: 20
区: 港区 page: 1 items: 20
区: 港区 page: 2 items: 20
区: 港区 page: 3 items: 20
区: 港区 page: 4 items: 20
区: 港区 page: 5 items: 20
区: 中央区 page: 1 items: 20
区: 中央区 page: 2 items: 20
区: 中央区 page: 3 items: 20
区: 中央区 page: 4 items: 20
区: 中央区 page: 5 items: 20
区: 世田谷区 page: 1 items: 20
区: 世田谷区 page: 2 items: 20
区: 世田谷区 page: 3 items: 20
区: 世田谷区 page: 4 items: 20
区: 世田谷区 page: 5 items: 20
区: 目黒区 page: 1 items: 20
区: 目黒区 page: 2 items: 20
区: 目黒区 page: 3 items: 20
区: 目黒区 page: 4 items: 20
区: 目黒区 page: 5 items: 20
区: 千代田区 page: 1 items: 20
区: 千代田区 page: 2 items: 20
区: 千代田区 page: 3 items: 20
区: 千代田区 page: 4 items: 20
区: 千代田区 page: 5 items: 20
合計 3607 件のデータを収集しました。
     区                       名称     カテゴリ         アドレス  \
0  新宿区  東京メトロ南北線 市ケ谷駅 12階建 築12年  賃貸マンション 

In [11]:
df.head()

,区,名称,カテゴリ,アドレス,アクセス,築年数,構造,階数,家賃,管理費,敷金,礼金,間取り,面積,物件画像URL,間取画像URL,物件詳細URL
0,新宿区,東京メトロ南北線 市ケ谷駅 12階建 築12年,賃貸マンション,東京都新宿区市谷本村町,"東京メトロ南北線/市ケ谷駅 歩5分, ＪＲ総武線/四ツ谷駅 歩7分, 都営新宿線/曙橋駅 歩10分",築12年,12階建,6階,11.2万円,8000円,11.2万円,11.2万円,1K,25.78m2,https://img01.suumo.com/front/gazo/fr/bukken/6...,https://img01.suumo.com/front/gazo/fr/bukken/5...,https://suumo.jp/chintai/jnc_000062882287/?bc=...
1,新宿区,東京メトロ南北線 市ケ谷駅 12階建 築12年,賃貸マンション,東京都新宿区市谷本村町,"東京メトロ南北線/市ケ谷駅 歩5分, ＪＲ総武線/四ツ谷駅 歩7分, 都営新宿線/曙橋駅 歩10分",築12年,12階建,5階,11万円,10000円,-,11万円,1K,25.6m2,https://img01.suumo.com/front/gazo/fr/bukken/6...,https://img01.suumo.com/front/gazo/fr/bukken/5...,https://suumo.jp/chintai/jnc_000062882287/?bc=...
2,新宿区,東京メトロ南北線 市ケ谷駅 12階建 築12年,賃貸マンション,東京都新宿区市谷本村町,"東京メトロ南北線/市ケ谷駅 歩5分, ＪＲ総武線/四ツ谷駅 歩7分, 都営新宿線/曙橋駅 歩10分",築12年,12階建,3階,11.5万円,8000円,11.5万円,11.5万円,1K,25.74m2,https://img01.suumo.com/front/gazo/fr/bukken/6...,https://img01.suumo.com/front/gazo/fr/bukken/5...,https://suumo.jp/chintai/jnc_000062882287/?bc=...
3,新宿区,東京メトロ南北線 市ケ谷駅 12階建 築12年,賃貸マンション,東京都新宿区市谷本村町,"東京メトロ南北線/市ケ谷駅 歩5分, ＪＲ総武線/四ツ谷駅 歩7分, 都営新宿線/曙橋駅 歩10分",築12年,12階建,5階,12.1万円,11000円,-,12.1万円,1K,25.62m2,https://img01.suumo.com/front/gazo/fr/bukken/6...,https://img01.suumo.com/front/gazo/fr/bukken/5...,https://suumo.jp/chintai/jnc_000062882287/?bc=...
4,新宿区,東京メトロ南北線 市ケ谷駅 12階建 築12年,賃貸マンション,東京都新宿区市谷本村町,"東京メトロ南北線/市ケ谷駅 歩5分, ＪＲ総武線/四ツ谷駅 歩7分, 都営新宿線/曙橋駅 歩10分",築12年,12階建,10階,12.3万円,11000円,-,12.3万円,1K,25.62m2,https://img01.suumo.com/front/gazo/fr/bukken/6...,https://img01.suumo.com/front/gazo/fr/bukken/5...,https://suumo.jp/chintai/jnc_000062882287/?bc=...


In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 対象の区とそのURLのリスト（HOMESの検索URLに合わせる）
wards = {
    "新宿区": "https://www.homes.co.jp/chintai/tokyo/shinjuku-city/?page={}",
    "渋谷区": "https://www.homes.co.jp/chintai/tokyo/shibuya-city/?page={}",
    "港区": "https://www.homes.co.jp/chintai/tokyo/minato-city/?page={}",
    "中央区": "https://www.homes.co.jp/chintai/tokyo/chuo-city/?page={}",
    "世田谷区": "https://www.homes.co.jp/chintai/tokyo/setagaya-city/?page={}",
    "目黒区": "https://www.homes.co.jp/chintai/tokyo/meguro-city/?page={}",
    "千代田区": "https://www.homes.co.jp/chintai/tokyo/chiyoda-city/?page={}"
}

max_page = 5  # 最大ページ数
all_data = []

for ward, base_url in wards.items():
    for page in range(1, max_page + 1):
        url = base_url.format(page)
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'lxml')
        items = soup.find_all("div", {"class": "module-cassette"})

        print("区:", ward, "page:", page, "items:", len(items))

        for item in items:
            base_data = {}
            base_data["区"] = ward
            base_data["名称"] = item.find("h2", {"class": "property-header-title"}).get_text(strip=True) if item.find("h2", {"class": "property-header-title"}) else None
            base_data["アドレス"] = item.find("p", {"class": "property-body-address"}).get_text(strip=True) if item.find("p", {"class": "property-body-address"}) else None
            base_data["アクセス"] = item.find("p", {"class": "property-body-traffic"}).get_text(strip=True) if item.find("p", {"class": "property-body-traffic"}) else None
            base_data["築年数"] = item.find("span", {"class": "property-year"}).get_text(strip=True) if item.find("span", {"class": "property-year"}) else None
            base_data["構造"] = item.find("span", {"class": "property-structure"}).get_text(strip=True) if item.find("span", {"class": "property-structure"}) else None

            room_items = item.find_all("div", {"class": "property-room"})

            for room_item in room_items:
                data = base_data.copy()
                data["階数"] = room_item.find("div", {"class": "property-room-floor"}).get_text(strip=True) if room_item.find("div", {"class": "property-room-floor"}) else None
                data["家賃"] = room_item.find("div", {"class": "property-room-rent"}).get_text(strip=True) if room_item.find("div", {"class": "property-room-rent"}) else None
                data["管理費"] = room_item.find("div", {"class": "property-room-fees"}).get_text(strip=True) if room_item.find("div", {"class": "property-room-fees"}) else None
                data["敷金"] = room_item.find("div", {"class": "property-room-deposit"}).get_text(strip=True) if room_item.find("div", {"class": "property-room-deposit"}) else None
                data["礼金"] = room_item.find("div", {"class": "property-room-keymoney"}).get_text(strip=True) if room_item.find("div", {"class": "property-room-keymoney"}) else None
                data["間取り"] = room_item.find("div", {"class": "property-room-layout"}).get_text(strip=True) if room_item.find("div", {"class": "property-room-layout"}) else None
                data["面積"] = room_item.find("div", {"class": "property-room-space"}).get_text(strip=True) if room_item.find("div", {"class": "property-room-space"}) else None
                data["物件詳細URL"] = room_item.find("a", {"class": "property-room-link"})['href'] if room_item.find("a", {"class": "property-room-link"}) else None

                all_data.append(data)

print(f"合計 {len(all_data)} 件のデータを収集しました。")

# データを確認する場合
df2 = pd.DataFrame(all_data)
print(df2.head())


区: 新宿区 page: 1 items: 0
区: 新宿区 page: 2 items: 0
区: 新宿区 page: 3 items: 0
区: 新宿区 page: 4 items: 0
区: 新宿区 page: 5 items: 0
区: 渋谷区 page: 1 items: 0
区: 渋谷区 page: 2 items: 0
区: 渋谷区 page: 3 items: 0
区: 渋谷区 page: 4 items: 0
区: 渋谷区 page: 5 items: 0
区: 港区 page: 1 items: 0
区: 港区 page: 2 items: 0
区: 港区 page: 3 items: 0
区: 港区 page: 4 items: 0
区: 港区 page: 5 items: 0
区: 中央区 page: 1 items: 0
区: 中央区 page: 2 items: 0
区: 中央区 page: 3 items: 0
区: 中央区 page: 4 items: 0
区: 中央区 page: 5 items: 0
区: 世田谷区 page: 1 items: 0
区: 世田谷区 page: 2 items: 0
区: 世田谷区 page: 3 items: 0
区: 世田谷区 page: 4 items: 0
区: 世田谷区 page: 5 items: 0
区: 目黒区 page: 1 items: 0
区: 目黒区 page: 2 items: 0
区: 目黒区 page: 3 items: 0
区: 目黒区 page: 4 items: 0
区: 目黒区 page: 5 items: 0
区: 千代田区 page: 1 items: 0
区: 千代田区 page: 2 items: 0
区: 千代田区 page: 3 items: 0
区: 千代田区 page: 4 items: 0
区: 千代田区 page: 5 items: 0
合計 0 件のデータを収集しました。
Empty DataFrame
Columns: []
Index: []


In [8]:
# 基本URLと最大ページ数の設定
base_url = "https://suumo.jp/chintai/tokyo/sc_shinjuku/?page={}"
max_page = 5  # 最大ページ数

all_data = []

for page in range(1, max_page + 1):
    url = base_url.format(page)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'lxml')
    items = soup.find_all("div", {"class": "cassetteitem"})


    print("page", page, "items", len(items))

    for item in items:
        base_data = {}
        base_data["名称"]     = item.find("div", {"class": "cassetteitem_content-title"}).get_text(strip=True) if item.find("div", {"class": "cassetteitem_content-title"}) else None
        base_data["カテゴリ"] = item.find("div", {"class": "cassetteitem_content-label"}).span.get_text(strip=True) if item.find("div", {"class": "cassetteitem_content-label"}) else None
        base_data["アドレス"] = item.find("li", {"class": "cassetteitem_detail-col1"}).get_text(strip=True) if item.find("li", {"class": "cassetteitem_detail-col1"}) else None
        
        # 駅のアクセス情報をまとめて取得
        base_data["アクセス"] = ", ".join([station.get_text(strip=True) for station in item.findAll("div", {"class": "cassetteitem_detail-text"})])

        construction_info = item.find("li", {"class": "cassetteitem_detail-col3"}).find_all("div") if item.find("li", {"class": "cassetteitem_detail-col3"}) else None
        base_data["築年数"] = construction_info[0].get_text(strip=True) if construction_info and len(construction_info) > 0 else None
        base_data["構造"] = construction_info[1].get_text(strip=True) if construction_info and len(construction_info) > 1 else None

        tbodys = item.find("table", {"class": "cassetteitem_other"}).findAll("tbody")

        for tbody in tbodys:
            data = base_data.copy()
            # 階数情報の正確な取得
            floor_info = tbody.find_all("td")[2].get_text(strip=True) if len(tbody.find_all("td")) > 2 else None
            data["階数"]   = floor_info
            data["家賃"]   = tbody.select_one(".cassetteitem_price--rent").get_text(strip=True) if tbody.select_one(".cassetteitem_price--rent") else None
            data["管理費"] = tbody.select_one(".cassetteitem_price--administration").get_text(strip=True) if tbody.select_one(".cassetteitem_price--administration") else None
            data["敷金"]   = tbody.select_one(".cassetteitem_price--deposit").get_text(strip=True) if tbody.select_one(".cassetteitem_price--deposit") else None
            data["礼金"]   = tbody.select_one(".cassetteitem_price--gratuity").get_text(strip=True) if tbody.select_one(".cassetteitem_price--gratuity") else None
            data["間取り"] = tbody.select_one(".cassetteitem_madori").get_text(strip=True) if tbody.select_one(".cassetteitem_madori") else None
            data["面積"]   = tbody.select_one(".cassetteitem_menseki").get_text(strip=True) if tbody.select_one(".cassetteitem_menseki") else None

            # 物件画像・間取り画像・詳細URLの取得を最後に行う #.img.get("rel")
            property_image_element = item.find(class_="cassetteitem_object-item")
            data["物件画像URL"] = property_image_element.img["rel"] if property_image_element and property_image_element.img else None

            floor_plan_image_element = item.find(class_="casssetteitem_other-thumbnail")
            data["間取画像URL"] = floor_plan_image_element.img["rel"] if floor_plan_image_element and floor_plan_image_element.img else None

            property_link_element = item.select_one("a[href*='/chintai/jnc_']")
            data["物件詳細URL"] = "https://suumo.jp" +property_link_element['href'] if property_link_element else None ## 不動産サイトから詳細URLリンクを読み解き作成

            all_data.append(data)


page 1 items 20
page 2 items 20
page 3 items 20
page 4 items 20
page 5 items 20


In [9]:
df = pd.DataFrame(all_data)
df = df.drop_duplicates() # 重複データの削除
df.head(2)

,名称,カテゴリ,アドレス,アクセス,築年数,構造,階数,家賃,管理費,敷金,礼金,間取り,面積,物件画像URL,間取画像URL,物件詳細URL
0,東京メトロ有楽町線 市ケ谷駅 12階建 築12年,賃貸マンション,東京都新宿区市谷本村町,"東京メトロ有楽町線/市ケ谷駅 歩5分, 東京メトロ南北線/市ケ谷駅 歩5分, ＪＲ総武線/四...",築12年,12階建,6階,11.2万円,8000円,11.2万円,11.2万円,1K,25.78m2,https://img01.suumo.com/front/gazo/fr/bukken/2...,https://img01.suumo.com/front/gazo/fr/bukken/5...,https://suumo.jp/chintai/jnc_000062882287/?bc=...
1,東京メトロ有楽町線 市ケ谷駅 12階建 築12年,賃貸マンション,東京都新宿区市谷本村町,"東京メトロ有楽町線/市ケ谷駅 歩5分, 東京メトロ南北線/市ケ谷駅 歩5分, ＪＲ総武線/四...",築12年,12階建,5階,11万円,10000円,-,11万円,1K,25.6m2,https://img01.suumo.com/front/gazo/fr/bukken/2...,https://img01.suumo.com/front/gazo/fr/bukken/5...,https://suumo.jp/chintai/jnc_000062882287/?bc=...


### ● Googleスプレッドシートへの書き込み・読み込み。

In [12]:
# google スプレッドシート 書き込み・読み込み
import gspread
from google.oauth2 import service_account
from google.oauth2.service_account import Credentials
from gspread_dataframe import get_as_dataframe
from gspread_dataframe import set_with_dataframe

In [13]:
#環境変数何やってもNG、原因不明
from dotenv import load_dotenv
import os

# 環境変数の読み込み
load_dotenv()

# 環境変数から認証情報を取得
SPREADSHEET_ID = os.getenv("SPREADSHEET_ID")
PRIVATE_KEY_PATH = os.getenv("PRIVATE_KEY_PATH")

In [14]:
print(os.getenv("PRIVATE_KEY_PATH"))

C:\Users\sator\OneDrive\Documents\Tech0-7-MOGI-PC-202312\03.Step3\Step3-1\不動産アプリ\mogi\service_account.json


In [15]:
PRIVATE_KEY_PATH

'C:\\Users\\sator\\OneDrive\\Documents\\Tech0-7-MOGI-PC-202312\\03.Step3\\Step3-1\\不動産アプリ\\mogi\\service_account.json'

In [16]:
#.envの前に、ファイルを認識しているか確認
file_path = './service_account.json'
if os.path.exists(file_path):
    print("ファイルが見つかりました。")
else:
    print("ファイルが見つかりません。")

ファイルが見つかりました。


##環境変数で成功##

In [18]:
# googleスプレッドシートの認証 jsonファイル読み込み(key値はGCPから取得)
#自分
SP_CREDENTIAL_FILE = PRIVATE_KEY_PATH

scopes = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
]

credentials = Credentials.from_service_account_file(
    SP_CREDENTIAL_FILE,
    scopes=scopes
)
gc = gspread.authorize(credentials)


SP_SHEET_KEY = SPREADSHEET_ID # d/〇〇/edit の〇〇部分
sh  = gc.open_by_key(SP_SHEET_KEY)

In [18]:
# 取得した不動産データの書き込み
SP_SHEET_wr     = 'suumo' # sheet名
worksheet_wr = sh.worksheet(SP_SHEET_wr) # シートのデータ取得
set_with_dataframe(worksheet_wr, df)

In [19]:
# 不動産データの取得
SP_SHEET     = 'suumo' # sheet名
worksheet = sh.worksheet(SP_SHEET) # シートのデータ取得
pre_data  = worksheet.get_all_values()
col_name = pre_data[0][:]
new_df = pd.DataFrame(pre_data[1:], columns=col_name) # 一段目をカラム、以下データフレームで取得

In [20]:
new_df.head(2)

,名称,カテゴリ,アドレス,アクセス,築年数,構造,階数,家賃,管理費,敷金,礼金,間取り,面積,物件画像URL,間取画像URL,物件詳細URL
0,東京メトロ有楽町線 市ケ谷駅 12階建 築12年,賃貸マンション,東京都新宿区市谷本村町,"東京メトロ有楽町線/市ケ谷駅 歩5分, 東京メトロ南北線/市ケ谷駅 歩5分, ＪＲ総武線/四...",築12年,12階建,6階,11.2万円,8000円,11.2万円,11.2万円,1K,25.78m2,https://img01.suumo.com/front/gazo/fr/bukken/2...,https://img01.suumo.com/front/gazo/fr/bukken/5...,https://suumo.jp/chintai/jnc_000062882287/?bc=...
1,東京メトロ有楽町線 市ケ谷駅 12階建 築12年,賃貸マンション,東京都新宿区市谷本村町,"東京メトロ有楽町線/市ケ谷駅 歩5分, 東京メトロ南北線/市ケ谷駅 歩5分, ＪＲ総武線/四...",築12年,12階建,5階,11万円,10000円,-,11万円,1K,25.6m2,https://img01.suumo.com/front/gazo/fr/bukken/2...,https://img01.suumo.com/front/gazo/fr/bukken/5...,https://suumo.jp/chintai/jnc_000062882287/?bc=...


In [21]:
new_df["築年数"]

0      築12年
1      築12年
2      築12年
3      築12年
4      築12年
       ... 
567    築18年
568    築18年
569    築17年
570    築17年
571    築17年
Name: 築年数, Length: 572, dtype: object

### ● 不動産データを加工する。

In [1]:
new_df.info()

NameError: name 'new_df' is not defined

In [23]:
#元
new_df["築年数"] = new_df["築年数"].apply( lambda x: 0 if x=="新築" else int(re.split("[築年]", x )[1]) )


In [24]:
def get_most_floor(x):
    if ('階建' not in x) :
        return np.nan
    elif('B' not in x) :
        list = re.findall(r'(\d+)階建',str(x))
        list = map(int, list)
        min_value = min(list)
        return min_value

new_df['構造'] = new_df['構造'].apply(get_most_floor)
print(new_df['構造'].head(5))

0    12
1    12
2    12
3    12
4    12
Name: 構造, dtype: int64


In [25]:
def get_floor(x):
    if ('階' not in x) :
        return np.nan
    elif('B' not in x) :
        list = re.findall(r'(\d+)階',str(x))
        # time_listを数値型に変換
        list = map(int, list)
        # time_listの最小値をmin_valueに代入
        min_value = min(list)
        return min_value
    else:
        list = re.findall(r'(\d+)階',str(x))
        # time_listを数値型に変換
        list = map(int, list)
        # time_listの最小値をmin_valueに代入
        min_value = -1*min(list)
        return min_value

new_df['階数'] = new_df['階数'].apply(get_floor)
print(new_df['階数'].head(5))

0     6
1     5
2     3
3    10
4     5
Name: 階数, dtype: int64


In [26]:
def change_fee(x):
    if ('万円' not in x) :
        return np.nan
    else:
        return float(x.split('万円')[0])

new_df['家賃'] = new_df['家賃'].apply(change_fee)
new_df['敷金'] = new_df['敷金'].apply(change_fee)
new_df['礼金'] = new_df['礼金'].apply(change_fee)

In [27]:
def change_fee2(x):
    if ('円' not in x) :
        return np.nan
    else:
        return float(x.split('円')[0])


new_df['管理費'] = new_df['管理費'].apply(change_fee2)

In [28]:
new_df['面積'] = new_df['面積'].apply(lambda x: float(x[:-2]))

In [29]:
new_df['区'] = new_df["アドレス"].apply(lambda x : x[x.find("都")+1:x.find("区")+1])

In [30]:
new_df['市町'] = new_df["アドレス"].apply(lambda x : x[x.find("区")+1 :-1])

In [31]:
def split_access(row):
    accesses = row['アクセス'].split(', ')
    results = {}

    for i, access in enumerate(accesses, start=1):
        if i > 3:
            break  # 最大3つのアクセス情報のみを考慮

        parts = access.split('/')
        if len(parts) == 2:
            line_station, walk = parts
            # ' 歩'で分割できるか確認
            if ' 歩' in walk:
                station, walk_min = walk.split(' 歩')
                # 歩数の分の数値だけを抽出
                walk_min = int(re.search(r'\d+', walk_min).group())
            else:
                station = None
                walk_min = None
        else:
            line_station = access
            station = walk_min = None

        results[f'アクセス①{i}線路名'] = line_station
        results[f'アクセス①{i}駅名'] = station
        results[f'アクセス①{i}徒歩(分)'] = walk_min

    return pd.Series(results)

# 新しい列をデータフレームに適用
new_df = new_df.join(new_df.apply(split_access, axis=1))

In [32]:
new_df.head(2)

,名称,カテゴリ,アドレス,アクセス,築年数,構造,階数,家賃,管理費,敷金,...,市町,アクセス①1線路名,アクセス①1駅名,アクセス①1徒歩(分),アクセス①2線路名,アクセス①2駅名,アクセス①2徒歩(分),アクセス①3線路名,アクセス①3駅名,アクセス①3徒歩(分)
0,東京メトロ有楽町線 市ケ谷駅 12階建 築12年,賃貸マンション,東京都新宿区市谷本村町,"東京メトロ有楽町線/市ケ谷駅 歩5分, 東京メトロ南北線/市ケ谷駅 歩5分, ＪＲ総武線/四...",12,12,6,11.2,8000.0,11.2,...,市谷本村,東京メトロ有楽町線,市ケ谷駅,5,東京メトロ南北線,市ケ谷駅,5.0,ＪＲ総武線,四ツ谷駅,8.0
1,東京メトロ有楽町線 市ケ谷駅 12階建 築12年,賃貸マンション,東京都新宿区市谷本村町,"東京メトロ有楽町線/市ケ谷駅 歩5分, 東京メトロ南北線/市ケ谷駅 歩5分, ＪＲ総武線/四...",12,12,5,11.0,10000.0,NaN,...,市谷本村,東京メトロ有楽町線,市ケ谷駅,5,東京メトロ南北線,市ケ谷駅,5.0,ＪＲ総武線,四ツ谷駅,8.0


In [33]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 572 entries, 0 to 571
Data columns (total 27 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   名称           572 non-null    object 
 1   カテゴリ         572 non-null    object 
 2   アドレス         572 non-null    object 
 3   アクセス         572 non-null    object 
 4   築年数          572 non-null    int64  
 5   構造           572 non-null    int64  
 6   階数           572 non-null    int64  
 7   家賃           572 non-null    float64
 8   管理費          535 non-null    float64
 9   敷金           409 non-null    float64
 10  礼金           323 non-null    float64
 11  間取り          572 non-null    object 
 12  面積           572 non-null    float64
 13  物件画像URL      572 non-null    object 
 14  間取画像URL      572 non-null    object 
 15  物件詳細URL      572 non-null    object 
 16  区            572 non-null    object 
 17  市町           572 non-null    object 
 18  アクセス①1線路名    572 non-null    object 
 19  アクセス①1駅名

In [34]:
# 取得した不動産データの書き込み
SP_SHEET_wr     = 'suumo2' # sheet名
worksheet_wr = sh.worksheet(SP_SHEET_wr) # シートのデータ取得
set_with_dataframe(worksheet_wr, new_df)

### ● 緯度経度情報取得

In [35]:
# 不動産データの取得
SP_SHEET     = 'suumo2' # sheet名
worksheet = sh.worksheet(SP_SHEET) # シートのデータ取得
pre_data  = worksheet.get_all_values()
col_name = pre_data[0][:]
new_df = pd.DataFrame(pre_data[1:], columns=col_name) # 一段目をカラム、以下データフレームで取得

In [36]:
#入ってなければインストール
pip install geopy

SyntaxError: invalid syntax (1025287607.py, line 2)

全物件をジオコーディングして緯度経度を取得する（すずゆうさん進化ver）

In [37]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
import pandas as pd
import time

# ジオコーダーの初期化
geolocator = Nominatim(user_agent="your_app_name", timeout=10)

current_count = 0
total_count = len(new_df['アドレス'])

# 住所から緯度と経度を取得する関数
def get_lat_lon(address, retries=3):
    global current_count
    current_count += 1
    
    for attempt in range(retries):
        try:
            location = geolocator.geocode(address)
            if location:
                print(f"{current_count}/{total_count} 件目実施中 結果: {location.latitude}, {location.longitude}")
                return location.latitude, location.longitude
            else:
                print(f"{current_count}/{total_count} 件目実施中 結果: 住所が見つかりません")
                return None, None
        except (GeocoderTimedOut, GeocoderServiceError) as e:
            print(f"Error retrieving location for address {address}: {e}. Retrying ({attempt + 1}/{retries})...")
            time.sleep(1)  # リトライの前に1秒待つ

    print(f"Failed to retrieve location for address {address} after {retries} retries")
    return None, None

# 新しい列を作成してデータフレームに追加
new_df['緯度'], new_df['経度'] = zip(*new_df['アドレス'].apply(get_lat_lon))

# 結果の表示
print(new_df.head())

# 単一の住所のテスト
address = "東京都江戸川区篠崎町７"
location = geolocator.geocode(address)
if location:
    print(f"テストアドレスの結果: {location.latitude}, {location.longitude}")
else:
    print("テストアドレスの結果: 住所が見つかりません")

1/572 件目実施中 結果: 35.6925023, 139.7299737
2/572 件目実施中 結果: 35.6925023, 139.7299737
3/572 件目実施中 結果: 35.6925023, 139.7299737
4/572 件目実施中 結果: 35.6925023, 139.7299737
5/572 件目実施中 結果: 35.6925023, 139.7299737
6/572 件目実施中 結果: 35.7027944, 139.71774460591615
7/572 件目実施中 結果: 35.7082189, 139.7122677
8/572 件目実施中 結果: 35.7082189, 139.7122677
9/572 件目実施中 結果: 35.7082189, 139.7122677
10/572 件目実施中 結果: 35.7082189, 139.7122677
11/572 件目実施中 結果: 35.7082189, 139.7122677
12/572 件目実施中 結果: 35.7082189, 139.7122677
13/572 件目実施中 結果: 35.7082189, 139.7122677
14/572 件目実施中 結果: 35.7082189, 139.7122677
15/572 件目実施中 結果: 35.7082189, 139.7122677
16/572 件目実施中 結果: 35.7082189, 139.7122677
17/572 件目実施中 結果: 35.7082189, 139.7122677
18/572 件目実施中 結果: 35.7082189, 139.7122677
19/572 件目実施中 結果: 35.7082189, 139.7122677
20/572 件目実施中 結果: 35.7082189, 139.7122677
21/572 件目実施中 結果: 35.7082189, 139.7122677
22/572 件目実施中 結果: 35.7082189, 139.7122677
23/572 件目実施中 結果: 35.715429650000004, 139.70274931100892
24/572 件目実施中 結果: 35.7095681, 139.6974063
25/

In [38]:
# 取得した不動産データの書き込み（最後に緯度経度まで入った物件情報を別sheetに出力）
SP_SHEET_wr     = 'suumo3' # sheet名
worksheet_wr = sh.worksheet(SP_SHEET_wr) # シートのデータ取得
set_with_dataframe(worksheet_wr, new_df)

以下は、元の石田コード。こっちでも動いたがアップデートのすずゆうver使うため、実行しない

In [61]:

from geopy.geocoders import Nominatim

# ジオコーダーの初期化
geolocator = Nominatim(user_agent="your_app_name")

current_count = 0
total_count = len(new_df['アドレス'])


# 住所から緯度と経度を取得する関数
def get_lat_lon(address):
    global current_count
    current_count += 1

    try:
        location = geolocator.geocode(address)
        print(f"{current_count}/{total_count} 件目実施中 結果: {location.latitude}, {location.longitude}")

        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except Exception as e:
        print(f"Error retrieving location for address {address}: {e}")
        return None, None

In [62]:
location = geolocator.geocode("東京都江戸川区篠崎町７")
print(location)

None


In [63]:
# 住所列があると仮定して、緯度経度列をデータフレームに追加
new_df['latitude'], new_df['longitude'] = zip(*new_df['アドレス'].apply(get_lat_lon))

1/572 件目実施中 結果: 35.7027944, 139.71774460591615
2/572 件目実施中 結果: 35.7031495, 139.7268033
3/572 件目実施中 結果: 35.7031495, 139.7268033
4/572 件目実施中 結果: 35.7031495, 139.7268033
5/572 件目実施中 結果: 35.7031495, 139.7268033
6/572 件目実施中 結果: 35.7031495, 139.7268033
7/572 件目実施中 結果: 35.7031495, 139.7268033
8/572 件目実施中 結果: 35.7031495, 139.7268033
9/572 件目実施中 結果: 35.7031495, 139.7268033
10/572 件目実施中 結果: 35.7031495, 139.7268033
Error retrieving location for address 東京都新宿区西新宿８: 'NoneType' object has no attribute 'latitude'
Error retrieving location for address 東京都新宿区西新宿８: 'NoneType' object has no attribute 'latitude'
Error retrieving location for address 東京都新宿区西新宿８: 'NoneType' object has no attribute 'latitude'
Error retrieving location for address 東京都新宿区西新宿８: 'NoneType' object has no attribute 'latitude'
Error retrieving location for address 東京都新宿区西新宿８: 'NoneType' object has no attribute 'latitude'
Error retrieving location for address 東京都新宿区西新宿８: 'NoneType' object has no attribute 'latitude'
Error retrieving

In [64]:
# 取得した不動産データの書き込み
SP_SHEET_wr     = 'suumo3' # sheet名
worksheet_wr = sh.worksheet(SP_SHEET_wr) # シートのデータ取得
set_with_dataframe(worksheet_wr, new_df)